# stop-to-stop segments losing curve
* when we cut it at particular stop locations along the `shape`, stringing those segments together as `shapely.geometry.LineString` forces those to be straight lines
* not ideal -- we need to keep all the points that are present in the `shape` so that the segments follow any curves or turns around streets

In [1]:
import dask_geopandas as dg
import dask.dataframe as dd
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely

from update_vars import SEGMENT_GCS, COMPILED_CACHED_VIEWS, analysis_date

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [ ]:
# Find the endpoint of that segment
# but also keep all the other shapely objects in between
# otherwise we lose the curves


In [8]:
test_shape1 = "b315bbbd7eff442b38f52615fde4a2d2"

test = dg.read_parquet(
    f"{SEGMENT_GCS}test_stop_segments_{analysis_date}"
).compute()

test = test[test.shape_array_key==test_shape1]

#test_shape2 = '9b91e8a8a48f69a5e935ae3781f773cc'

df = gpd.read_parquet(
    f"./scripts/data/stops_projected_wide.parquet", 
    filters = [[("shape_array_key", "==", test_shape1)]]
)

stops = gpd.read_parquet(
    f"./scripts/data/stops_projected.parquet",
    filters = [[("shape_array_key", "==", test_shape1)]]
)

In [6]:
import folium

In [9]:
test.explore('segment_sequence',
    tiles='CartoDB Positron', categorical=True)

In [10]:
m = test.explore(
    "segment_sequence", tiles="CartoDB Positron", 
    name = "segments", legend=False,
    categorical = True, cmap = "tab20",
    highlight = True,
    highlight_kwds = {'fillColor': '#DD1C77',"fillOpacity": 0.6},
)

stops.explore("stop_sequence", m=m, categorical = False, name="stops")
folium.LayerControl().add_to(m)
m